# 알림 판단 에이전트

시간(달력)과 컨텍스트를 이용해서 알림을 보낼지 말지 판단하는 에이전트

In [19]:
# 필요한 라이브러리 import
from datetime import datetime, timedelta
from typing import TypedDict, Literal, Optional, List
from langgraph.graph import StateGraph, END
import os
import json
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage

load_dotenv()
api_key = os.getenv("UPSTAGE_API_KEY")

# LLM 초기화
chat = ChatUpstage(
    model="solar-pro2-251215",
    upstage_api_key=api_key
)

# 참고: extractor.ipynb의 extract_diary_info 함수를 사용하여 일기에서 정보를 추출할 수 있습니다
# from extractor import extract_diary_info
# extracted = extract_diary_info("일기 내용")

In [20]:
### State 정의

class NotificationPlanningState(TypedDict):
    """알림 계획 에이전트의 State"""
    current_time: str  # "YYYY-MM-DD HH:mm:ss"
    calendar_events: List[dict]  # [{"date": "...", "title": "...", "type": "..."}]
    diary_entries: List[dict]  # extractor로 분석된 일기 항목 [{"date": "...", "content": "...", "topic": "...", "emotion": "..."}]
    messages: List[dict]  # 사용자가 직접 작성한 원본 메시지 (extractor 분석 안 함) [{"content": "...", "datetime": "..."}]
    new_diary_entry: dict  # 방금 작성된 새로운 일기 (extractor로 분석된 결과 포함)
    # {"content": "...", "datetime": "...", "topic": "...", "emotion": "..."}
    should_send: Optional[bool]  # 알림 전송 여부
    send_time: Optional[str]  # 알림 전송 시간
    message: Optional[str]  # 알림 메시지
    reason: Optional[str]  # 판단 사유

In [21]:
### 노드 함수 정의

def analyze_context(state: NotificationPlanningState) -> NotificationPlanningState:
    """컨텍스트 분석 노드 - 새로운 일기와 모든 과거 메시지, 시간, 달력을 종합 분석"""
    current_time = datetime.fromisoformat(state["current_time"])
    
    # 새로운 일기 정보 (extractor로 분석된 결과 포함)
    new_entry = state["new_diary_entry"]
    new_content = new_entry.get("content", "") if isinstance(new_entry, dict) else ""
    new_datetime = new_entry.get("datetime", state["current_time"]) if isinstance(new_entry, dict) else state["current_time"]
    new_topic = new_entry.get("topic", "") if isinstance(new_entry, dict) else ""
    new_emotion = new_entry.get("emotion", "") if isinstance(new_entry, dict) else ""
    
    # 과거 메시지들 (모든 일기 히스토리)
    past_messages_count = len(state["messages"])
    
    print(f"[analyze_context] 새로운 일기 작성 시점 분석")
    print(f"  - 새로운 일기: {new_content[:50]}...")
    print(f"  - 주제: {new_topic}")
    print(f"  - 감정: {new_emotion}")
    print(f"  - 일기 작성 시간: {new_datetime}")
    print(f"  - 현재 시간: {current_time}")
    print(f"  - 과거 일기/메시지 수: {past_messages_count}개")
    print(f"  - 달력 이벤트 수: {len(state['calendar_events'])}개")
    print(f"  - 일기 항목 수: {len(state['diary_entries'])}개")
    
    return state

def check_time_context(state: NotificationPlanningState) -> NotificationPlanningState:
    """시간 컨텍스트 체크 - 현재 시간과 달력 이벤트를 고려"""
    current_time = datetime.fromisoformat(state["current_time"])
    
    # 오늘 날짜
    today = current_time.date()
    
    # 오늘의 달력 이벤트 확인
    today_events = [
        event for event in state["calendar_events"]
        if datetime.fromisoformat(event["date"]).date() == today
    ]
    
    # 시간대 체크 (예: 오전 9시~오후 10시 사이만 알림 가능)
    hour = current_time.hour
    is_appropriate_time = 9 <= hour <= 22
    
    print(f"[check_time_context] 시간 컨텍스트 체크")
    print(f"  - 현재 시간: {current_time.strftime('%Y-%m-%d %H:%M')}")
    print(f"  - 적절한 시간대: {is_appropriate_time}")
    print(f"  - 오늘의 이벤트: {len(today_events)}개")
    
    return state

def decide_notification(state: NotificationPlanningState) -> NotificationPlanningState:
    """알림 여부 결정 노드 - 모든 일기/메시지를 종합하여 LLM으로 판단"""
    current_time = datetime.fromisoformat(state["current_time"])
    
    # 새로운 일기 정보 (extractor로 분석된 결과 포함)
    new_entry = state["new_diary_entry"]
    new_content = new_entry.get("content", "") if isinstance(new_entry, dict) else ""
    new_datetime = new_entry.get("datetime", state["current_time"]) if isinstance(new_entry, dict) else state["current_time"]
    new_entry_time = datetime.fromisoformat(new_datetime) if isinstance(new_datetime, str) else current_time
    new_topic = new_entry.get("topic", "") if isinstance(new_entry, dict) else ""
    new_emotion = new_entry.get("emotion", "") if isinstance(new_entry, dict) else ""
    
    # 오늘의 달력 이벤트
    today = current_time.date()
    today_events = [
        event for event in state["calendar_events"]
        if datetime.fromisoformat(event["date"]).date() == today
    ]
    
    # 과거 메시지들을 요약 (최근 N개만 포함하여 프롬프트 길이 제한)
    # messages는 원본 메시지이므로 topic, emotion 없음
    recent_messages = state["messages"][-10:] if len(state["messages"]) > 10 else state["messages"]
    past_messages_summary = "\n".join([
        f"- {msg.get('datetime', '')}: {msg.get('content', '')[:80]}"
        for msg in recent_messages
    ]) if recent_messages else "없음"
    
    # 일기 항목 요약 (extractor로 분석된 결과이므로 topic, emotion 포함)
    diary_summary = "\n".join([
        f"- {entry.get('date', '')}: [{entry.get('topic', 'N/A')}] [{entry.get('emotion', 'N/A')}] - {entry.get('content', '')[:50]}"
        for entry in state["diary_entries"][-5:]  # 최근 5개만
    ]) if state["diary_entries"] else "없음"
    
    # LLM 호출을 위한 프롬프트 구성
    prompt = f"""사용자가 새로운 일기를 작성했습니다. 과거의 모든 일기와 메시지를 종합하여 회고를 유도하기 위한 알림 전송 여부를 판단하고 JSON으로 응답하세요.

=== 새로운 일기 (분석 결과 포함) ===
내용: {new_content}
주제: {new_topic}
감정: {new_emotion}
작성 시간: {new_entry_time.strftime('%Y-%m-%d %H:%M:%S')}

=== 현재 상황 ===
현재 시간: {current_time.strftime('%Y-%m-%d %H:%M:%S')}
오늘의 달력 이벤트: {len(today_events)}개
  {chr(10).join([f"- {e.get('title', '')} ({e.get('date', '')})" for e in today_events[:3]])}

=== 과거 원본 메시지 (최근 {len(recent_messages)}개) ===
{past_messages_summary}

=== 분석된 일기 항목 (최근 5개, extractor로 분석됨) ===
{diary_summary}

=== 판단 기준 ===
1. 새로운 일기가 작성된 시점에서 과거 일기들을 종합적으로 고려
2. 회고를 유도할 만한 적절한 타이밍을 생각하세요. 다음 상황에서는 회고 유도가 좋습니다:
   - 불안하거나 스트레스가 많은 이벤트(회의, 발표, 면접 등)가 끝난 직후
   - 힘든 일이 있어서 아무것도 못하고 있을 때 (부정적 감정이 지속되는 경우)
   - 캘린더에 중요한 이벤트가 있었고, 그 이벤트가 끝난 후
   - 과거 일기에서 비슷한 패턴(같은 주제, 같은 감정)이 반복될 때
   - 감정 변화가 큰 시점 (부정적 → 긍정적, 또는 그 반대)
3. 사용자의 감정 상태와 일기 패턴을 고려

JSON 형식:
{{
  "should_send": true/false,
  "send_time": "YYYY-MM-DD HH:MM:SS",
  "reason": "판단 사유 (모든 일기를 종합한 근거 포함)"
}}"""
    
    # LLM 호출
    response = chat.invoke(prompt)
    
    # JSON 파싱
    try:
        # 응답에서 JSON 추출
        response_text = response.content.strip()
        
        # JSON 부분만 추출 (```json ... ``` 또는 {...} 형식)
        if "```json" in response_text:
            json_start = response_text.find("```json") + 7
            json_end = response_text.find("```", json_start)
            response_text = response_text[json_start:json_end].strip()
        elif "```" in response_text:
            json_start = response_text.find("```") + 3
            json_end = response_text.find("```", json_start)
            response_text = response_text[json_start:json_end].strip()
        
        # 첫 번째 JSON 객체만 추출 (중괄호로 시작하는 부분)
        if "{" in response_text:
            start_idx = response_text.find("{")
            brace_count = 0
            end_idx = start_idx
            
            for i in range(start_idx, len(response_text)):
                if response_text[i] == "{":
                    brace_count += 1
                elif response_text[i] == "}":
                    brace_count -= 1
                    if brace_count == 0:
                        end_idx = i + 1
                        break
            
            response_text = response_text[start_idx:end_idx]
        
        result_json = json.loads(response_text)
        should_send = result_json.get("should_send", False)
        send_time = result_json.get("send_time", current_time.strftime("%Y-%m-%d %H:%M:%S"))
        reason = result_json.get("reason", "판단 완료")
        
    except Exception as e:
        print(f"[decide_notification] JSON 파싱 실패: {e}")
        print(f"[decide_notification] 원본 응답: {response.content[:200]}...")
        # 기본값 사용
        should_send = False
        send_time = (current_time + timedelta(days=1)).replace(hour=9, minute=0, second=0).strftime("%Y-%m-%d %H:%M:%S")
        reason = "JSON 파싱 실패로 인한 기본값"
    
    state["should_send"] = should_send
    state["send_time"] = send_time
    state["reason"] = reason
    if should_send:
        state["message"] = f"회고 알림: 과거 일기들을 돌아보며 회고해보세요"
    else:
        state["message"] = None
    
    print(f"[decide_notification] 종합 판단 결과")
    print(f"  - 알림 전송: {should_send}")
    if should_send:
        print(f"  - 전송 시간: {send_time}")
        print(f"  - 알림 메시지: {state['message']}")
    print(f"  - 사유: {reason}")
    
    return state

In [22]:
### 라우팅 함수 정의 (단순화 - 더 이상 필요 없음)
# 새로운 일기 작성 시점에 한 번만 판단하므로 라우팅이 필요 없음

In [23]:
### 그래프 구성

# StateGraph 생성
workflow = StateGraph(NotificationPlanningState)

# 노드 추가
workflow.add_node("analyze_context", analyze_context)
workflow.add_node("check_time_context", check_time_context)
workflow.add_node("decide_notification", decide_notification)

# 엣지 추가 - 단순 선형 흐름 (새 일기 작성 시 한 번만 실행)
workflow.set_entry_point("analyze_context")
workflow.add_edge("analyze_context", "check_time_context")
workflow.add_edge("check_time_context", "decide_notification")
workflow.add_edge("decide_notification", END)

# 그래프 컴파일
app = workflow.compile()

print("✅ 알림 판단 에이전트가 생성되었습니다!")
print("   - 새로운 일기 작성 시 모든 과거 일기를 종합하여 판단합니다.")

✅ 알림 판단 에이전트가 생성되었습니다!
   - 새로운 일기 작성 시 모든 과거 일기를 종합하여 판단합니다.


In [24]:
### 테스트 실행

# 초기 state 생성 - 새로운 일기가 작성된 시점 시뮬레이션
initial_state = NotificationPlanningState(
    current_time=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    calendar_events=[
        {"date": "2026-01-15 09:00:00", "title": "팀 미팅", "type": "meeting"},
        {"date": "2026-01-16 10:00:00", "title": "프로젝트 발표", "type": "presentation"},
    ],
    # extractor로 분석된 일기 항목 (topic, emotion 포함)
    diary_entries=[
        {"date": "2026-01-14", "content": "오늘 하루가 힘들었어요", "topic": "일상", "emotion": "sad"},
    ],
    # 사용자가 직접 작성한 원본 메시지 (content, datetime만)
    messages=[
        {"content": "아 오늘 회의 사장 개빡침", "datetime": "2026-01-12 10:30:00"},
        {"content": "야식 먹어서 살찌겟네 ㅠ", "datetime": "2026-01-12 22:15:00"},
        {"content": "내일 친구 5시 고깃집", "datetime": "2026-01-12 15:45:00"},
        {"content": "내일 회의 있음", "datetime": "2026-01-13 09:20:00"},
    ],
    # 방금 작성된 새로운 일기 (extractor로 분석된 결과 포함)
    # 실제 사용 시에는 extractor.ipynb의 extract_diary_info 함수를 사용하여 추출
    new_diary_entry={
        "content": "오늘은 프로젝트 발표 준비를 했다. 조금 긴장되지만 잘 할 수 있을 것 같다.",
        "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "topic": "프로젝트 발표",
        "emotion": "긴장"
    },
    should_send=None,
    send_time=None,
    message=None,
    reason=None
)

# 워크플로우 실행
result = app.invoke(initial_state)

print("\n✅ 워크플로우 실행 완료!")
print(f"\n=== 종합 판단 결과 ===")
print(f"  - 새로운 일기 작성 시간: {result['new_diary_entry'].get('datetime', '')}")
print(f"  - 과거 일기/메시지 수: {len(result['messages'])}개")
print(f"  - 알림 전송 여부: {result['should_send']}")
if result['should_send']:
    print(f"  - 전송 시간: {result['send_time']}")
    print(f"  - 알림 메시지: {result['message']}")
print(f"  - 판단 사유: {result['reason']}")

[analyze_context] 새로운 일기 작성 시점 분석
  - 새로운 일기: 오늘은 프로젝트 발표 준비를 했다. 조금 긴장되지만 잘 할 수 있을 것 같다....
  - 주제: 프로젝트 발표
  - 감정: 긴장
  - 일기 작성 시간: 2026-01-14 15:07:56
  - 현재 시간: 2026-01-14 15:07:56
  - 과거 일기/메시지 수: 4개
  - 달력 이벤트 수: 2개
  - 일기 항목 수: 1개
[check_time_context] 시간 컨텍스트 체크
  - 현재 시간: 2026-01-14 15:07
  - 적절한 시간대: True
  - 오늘의 이벤트: 0개
[decide_notification] 종합 판단 결과
  - 알림 전송: False
  - 사유: 현재 상황 분석 결과 회고 유도가 필요하지 않습니다. 판단 근거는 다음과 같습니다:
1. 새로 작성된 일기(프로젝트 발표 준비)에서 표현된 '긴장' 감정은 예정된 이벤트에 대한 자연스러운 반응이며, 아직 발표 자체가 완료되지 않은 상태입니다. 회고 유도는 이벤트 종료 후에 적합합니다.
2. 최근 5일간의 일기를 종합할 때, 부정적 감정(sad)과 긴장 감정이 혼재되어 있으나 패턴을 반복하거나 감정 변화가 급격히 나타나는 상황은 아닙니다.
3. 현재 캘린더에 등록된 중요한 이벤트가 없으며, 과거 메시지에서도 발표와 직접적으로 연관된 사전 준비 상황에 대한 기록이 없습니다.
4. 가장 최근의 부정적 감정 일기(2026-01-14)와 새로운 일기의 감정/주제가 명확히 구분되므로, 추가적인 심리적 간섭이 필요하지 않은 단계로 판단됩니다.

따라서 프로젝트 발표 완료 후에 회고를 유도하는 것이 더 효과적일 것으로 예상됩니다.

✅ 워크플로우 실행 완료!

=== 종합 판단 결과 ===
  - 새로운 일기 작성 시간: 2026-01-14 15:07:56
  - 과거 일기/메시지 수: 4개
  - 알림 전송 여부: False
  - 판단 사유: 현재 상황 분석 결과 회고 유도가 필요하지 않습

### 회고 유도 좋은 타이밍 예시

In [25]:
### 예시 1: 불안한 이벤트(발표)가 끝난 후

# 발표가 끝난 직후 - 회고 유도 좋은 타이밍
example1_state = NotificationPlanningState(
    current_time="2026-01-16 15:00:00",  # 발표가 끝난 직후
    calendar_events=[
        {"date": "2026-01-16 10:00:00", "title": "프로젝트 발표", "type": "presentation"},  # 오늘 발표가 있었음
    ],
    diary_entries=[
        {"date": "2026-01-15", "content": "내일 발표 준비해야 하는데 불안하다", "topic": "프로젝트 발표", "emotion": "불안"},
        {"date": "2026-01-14", "content": "발표 준비하느라 밤샜다", "topic": "프로젝트 발표", "emotion": "긴장"},
    ],
    messages=[
        {"content": "발표 준비 너무 힘들다", "datetime": "2026-01-15 20:00:00"},
        {"content": "불안해서 잠이 안 온다", "datetime": "2026-01-15 23:00:00"},
    ],
    new_diary_entry={
        "content": "발표 끝났다. 생각보다 괜찮았어",
        "datetime": "2026-01-16 14:30:00",
        "topic": "프로젝트 발표",
        "emotion": "안도"
    },
    should_send=None,
    send_time=None,
    message=None,
    reason=None
)

print("=" * 60)
print("예시 1: 불안한 이벤트(발표)가 끝난 후")
print("=" * 60)
print("상황: 프로젝트 발표가 방금 끝났고, 발표 전 불안했던 일기들이 있음")
print("예상: 회고 유도 알림 전송 (발표 전후 감정 변화를 돌아볼 좋은 타이밍)")

result1 = app.invoke(example1_state)
print(f"\n알림 전송: {result1['should_send']}")
if result1['should_send']:
    print(f"전송 시간: {result1['send_time']}")
    print(f"사유: {result1['reason']}")

예시 1: 불안한 이벤트(발표)가 끝난 후
상황: 프로젝트 발표가 방금 끝났고, 발표 전 불안했던 일기들이 있음
예상: 회고 유도 알림 전송 (발표 전후 감정 변화를 돌아볼 좋은 타이밍)
[analyze_context] 새로운 일기 작성 시점 분석
  - 새로운 일기: 발표 끝났다. 생각보다 괜찮았어...
  - 주제: 프로젝트 발표
  - 감정: 안도
  - 일기 작성 시간: 2026-01-16 14:30:00
  - 현재 시간: 2026-01-16 15:00:00
  - 과거 일기/메시지 수: 2개
  - 달력 이벤트 수: 1개
  - 일기 항목 수: 2개
[check_time_context] 시간 컨텍스트 체크
  - 현재 시간: 2026-01-16 15:00
  - 적절한 시간대: True
  - 오늘의 이벤트: 1개
[decide_notification] 종합 판단 결과
  - 알림 전송: True
  - 전송 시간: 2026-01-16 15:00:00
  - 알림 메시지: 회고 알림: 과거 일기들을 돌아보며 회고해보세요
  - 사유: 회고를 유도하는 알림을 전송하기 적합한 상황입니다. 판단 근거는 다음과 같습니다:

1. 과거 일기에서 동일한 주제(프로젝트 발표)에 대해 지속적으로 [불안] [긴장] 등 부정적 감정을 기록했으나, 새로운 일기에서는 [안도]라는 긍정적 감정 변화가 나타났습니다. 이는 감정 변화가 큰 시점에 해당합니다.

2. 오늘의 캘린더 이벤트(프로젝트 발표 10:00)가 종료된 직후(14:30) 새로운 일기가 작성되었으며, 이벤트 후 사용자의 감정 상태를 회고할 적절한 타이밍입니다.

3. 최근 2일간의 메시지에서도 발표에 대한 강한 스트레스와 불안이 지속적으로 나타났으며, 이는 현재 일기와 대조되어 회고를 통해 성장 경험을 정리하기에 적합합니다.

4. 동일한 주제(프로젝트 발표)가 최근 5개 일기 중 3번 등장하며 패턴이 반복되고 있어, 이를 종합적으로 되돌아볼 필요가 있습니다.

알림 전송: True
전송 시간: 2026-01

In [26]:
### 예시 2: 힘든 일 후 아무것도 못하고 있을 때

# 힘든 일이 있어서 부정적 감정이 지속되는 경우
example2_state = NotificationPlanningState(
    current_time="2026-01-17 18:00:00",  # 저녁 시간
    calendar_events=[],  # 오늘 특별한 일정 없음
    diary_entries=[
        {"date": "2026-01-15", "content": "부장님한테 혼났다", "topic": "부장회의", "emotion": "빡침"},
        {"date": "2026-01-16", "content": "오늘도 힘들었다", "topic": "일상", "emotion": "슬픔"},
    ],
    messages=[
        {"content": "아 오늘 회의 사장 개빡침", "datetime": "2026-01-15 10:30:00"},
        {"content": "너무 힘들어서 아무것도 못하겠다", "datetime": "2026-01-16 19:00:00"},
        {"content": "그냥 누워있고 싶다", "datetime": "2026-01-17 15:00:00"},
    ],
    new_diary_entry={
        "content": "오늘도 아무것도 못했다. 그냥 누워있었다",
        "datetime": "2026-01-17 17:30:00",
        "topic": "일상",
        "emotion": "무기력"
    },
    should_send=None,
    send_time=None,
    message=None,
    reason=None
)

print("\n" + "=" * 60)
print("예시 2: 힘든 일 후 아무것도 못하고 있을 때")
print("=" * 60)
print("상황: 연속으로 힘든 일이 있었고, 현재 무기력한 상태")
print("예상: 회고 유도 알림 전송 (과거 일기들을 돌아보며 감정을 정리할 좋은 타이밍)")

result2 = app.invoke(example2_state)
print(f"\n알림 전송: {result2['should_send']}")
if result2['should_send']:
    print(f"전송 시간: {result2['send_time']}")
    print(f"사유: {result2['reason']}")


예시 2: 힘든 일 후 아무것도 못하고 있을 때
상황: 연속으로 힘든 일이 있었고, 현재 무기력한 상태
예상: 회고 유도 알림 전송 (과거 일기들을 돌아보며 감정을 정리할 좋은 타이밍)
[analyze_context] 새로운 일기 작성 시점 분석
  - 새로운 일기: 오늘도 아무것도 못했다. 그냥 누워있었다...
  - 주제: 일상
  - 감정: 무기력
  - 일기 작성 시간: 2026-01-17 17:30:00
  - 현재 시간: 2026-01-17 18:00:00
  - 과거 일기/메시지 수: 3개
  - 달력 이벤트 수: 0개
  - 일기 항목 수: 2개
[check_time_context] 시간 컨텍스트 체크
  - 현재 시간: 2026-01-17 18:00
  - 적절한 시간대: True
  - 오늘의 이벤트: 0개
[decide_notification] 종합 판단 결과
  - 알림 전송: True
  - 전송 시간: 2026-01-17 18:05:00
  - 알림 메시지: 회고 알림: 과거 일기들을 돌아보며 회고해보세요
  - 사유: 다음 기준에 따라 회고 알림 전송이 필요하다고 판단되었습니다: 
1. 감정 지속성: 최근 3일치 메시지(1/15-1/17)와 분석된 일기(1/15-1/16)에서 '무기력', '빡침', '슬픔' 등 부정적 감정이 계속해서 관찰됨. 특히 '아무것도 못했다'는 표현이 1/16과 1/17 일기 모두에서 반복됨 
2. 패턴 분석: '일상' 주체에 대해 무기력/슬픔 감정이 2회 연속으로 기록되었으며, 과거 메시지에서도 유사한 무기력 상태(눕고 싶다/아무것도 못하겠다)가 확인됨 
3. 상황적 요소: 오늘 달력 이벤트가 없음에도 지속적으로 무기력한 상태를 보임. 이는 외부 스트레스 요인보다 내적 상태 고착화 가능성을 시사 
4. 회고 필요성: 3일 연속 부정적 감정 지속 시 자기반성 유도가 긍정적 행동 변화로 이어질 수 있음. 특히 '아무것도 못했다'는 자기비하적 표현에 대한 인지적 재구성(cognitive restructuring

In [27]:
### 예시 3: 캘린더 이벤트(면접) 후 회고 타이밍

# 면접 같은 중요한 이벤트가 끝난 후
example3_state = NotificationPlanningState(
    current_time="2026-01-18 16:00:00",  # 면접이 끝난 직후
    calendar_events=[
        {"date": "2026-01-18 14:00:00", "title": "회사 면접", "type": "interview"},  # 오늘 면접이 있었음
    ],
    diary_entries=[
        {"date": "2026-01-17", "content": "내일 면접 너무 떨린다", "topic": "면접", "emotion": "불안"},
        {"date": "2026-01-16", "content": "면접 준비하느라 밤샜다", "topic": "면접", "emotion": "긴장"},
    ],
    messages=[
        {"content": "면접 준비 너무 힘들다", "datetime": "2026-01-17 22:00:00"},
        {"content": "불안해서 잠이 안 온다", "datetime": "2026-01-17 23:30:00"},
    ],
    new_diary_entry={
        "content": "면접 끝났다. 생각보다 괜찮았는데 결과가 걱정된다",
        "datetime": "2026-01-18 15:30:00",
        "topic": "면접",
        "emotion": "걱정"
    },
    should_send=None,
    send_time=None,
    message=None,
    reason=None
)

print("\n" + "=" * 60)
print("예시 3: 캘린더 이벤트(면접) 후 회고 타이밍")
print("=" * 60)
print("상황: 면접이 방금 끝났고, 면접 전 불안했던 일기들이 있음")
print("예상: 회고 유도 알림 전송 (면접 전후 감정과 경험을 돌아볼 좋은 타이밍)")

result3 = app.invoke(example3_state)
print(f"\n알림 전송: {result3['should_send']}")
if result3['should_send']:
    print(f"전송 시간: {result3['send_time']}")
    print(f"사유: {result3['reason']}")


예시 3: 캘린더 이벤트(면접) 후 회고 타이밍
상황: 면접이 방금 끝났고, 면접 전 불안했던 일기들이 있음
예상: 회고 유도 알림 전송 (면접 전후 감정과 경험을 돌아볼 좋은 타이밍)
[analyze_context] 새로운 일기 작성 시점 분석
  - 새로운 일기: 면접 끝났다. 생각보다 괜찮았는데 결과가 걱정된다...
  - 주제: 면접
  - 감정: 걱정
  - 일기 작성 시간: 2026-01-18 15:30:00
  - 현재 시간: 2026-01-18 16:00:00
  - 과거 일기/메시지 수: 2개
  - 달력 이벤트 수: 1개
  - 일기 항목 수: 2개
[check_time_context] 시간 컨텍스트 체크
  - 현재 시간: 2026-01-18 16:00
  - 적절한 시간대: True
  - 오늘의 이벤트: 1개
[decide_notification] 종합 판단 결과
  - 알림 전송: True
  - 전송 시간: 2026-01-18 16:30:00
  - 알림 메시지: 회고 알림: 과거 일기들을 돌아보며 회고해보세요
  - 사유: 회고 유도가 필요한 상황으로 판단됩니다. 판단 근거는 다음과 같습니다:

1. 과거 일기 분석(2026-01-16~2026-01-17)에서 '면접'과 관련된 [불안], [긴장] 감정이 지속적으로 나타났으며, 최근 메시지(2026-01-17)에서도 동일한 스트레스 패턴이 관찰됨
2. 오늘 캘린더 이벤트(14:00 면접)가 종료된 직후(작성 시간 15:30) 일기가 작성되었고, 이는 기준 2번(불안한 이벤트 직후)과 3번(중요 이벤트 종료 후)에 정확히 부합
3. 현재 감정(걱정)이 과거 감정 패턴과 유사하지만, '생각보다 괜찮았는데'라는 표현에서 감정 변화 가능성(긴장 → 부분적 안도)이 확인되어 회고를 통해 사고를 정리할 필요가 있음
4. 3일 연속으로 면접 관련 스트레스가 기록된 점을 고려할 때, 회고를 통해 스트레스 원인을 분석하고 대처 방안을 모색할 적절한 시점임

알림 전송: True
전송 시간: 202

### Extractor와 연동 예시

실제 사용 시에는 extractor를 먼저 실행하여 일기에서 정보를 추출한 후, 그 결과를 plan에 전달합니다.

In [28]:
# Extractor와 Plan 연동 예시
# 실제 사용 시에는 다음과 같이 사용합니다:

# 1. extractor.ipynb에서 extract_diary_info 함수 import (또는 직접 실행)
# from extractor import extract_diary_info

# 2. 새로운 일기 작성 시 정보 추출
# new_diary_content = "아 부장 ㅅㅂ 화나네 회의때깨짐"
# extracted_info = extract_diary_info(new_diary_content)
# print(f"추출된 정보: {extracted_info}")
# # 출력 예: {'topic': '부장회의', 'emotion': '빡침', 'datetime': '...'}

# 3. 추출된 정보를 포함하여 plan 실행
# new_diary_entry = {
#     "content": new_diary_content,
#     "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
#     **extracted_info  # topic, emotion 포함
# }

# 4. plan 실행
# initial_state = NotificationPlanningState(
#     current_time=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
#     calendar_events=[],
#     diary_entries=[],  # extractor로 분석된 일기 항목들 (topic, emotion 포함)
#     messages=[],  # 사용자가 직접 작성한 원본 메시지 (content, datetime만)
#     new_diary_entry=new_diary_entry,  # extractor로 분석된 결과 포함
#     should_send=None,
#     send_time=None,
#     message=None,
#     reason=None
# )
# result = app.invoke(initial_state)

print("✅ Extractor와 Plan 연동 방법이 준비되었습니다!")

✅ Extractor와 Plan 연동 방법이 준비되었습니다!
